# [prompts](https://python.langchain.com/docs/modules/model_io/prompts/)
语言模型的提示是由用户提供的一组指令或输入，用于指导模型的响应，帮助它理解上下文并生成相关且一致的基于语言的输出，例如回答问题、完成句子或参与对话。



In [2]:
from dotenv import load_dotenv, find_dotenv
from langchain.globals import set_debug
load_dotenv(find_dotenv())
set_debug(False)

## 示例
这个快速入门提供了如何使用提示的基本概述。

提示模板是为语言模型生成提示的预定义配方。

模板可能包括说明、少量示例以及适合给定任务的特定上下文和问题。

LangChain提供了创建和使用提示模板的工具。

LangChain努力创建与模型无关的模板，以便在不同的语言模型之间轻松重用现有模板。

通常，语言模型期望提示符要么是字符串，要么是聊天消息列表。

### PromptTemplate
使用`PromptTemplate`为字符串提示创建一个模板。默认情况下，`PromptTemplate`使用Python的`str.format`语法进行模板化。

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(input_variables=["adjective", "content"], 
                                 template= "Tell me a {adjective} joke about {content}.")

prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.")
prompt_template.format(adjective="funny", content="chickens")


'Tell me a funny joke about chickens.'

模板支持任意数量的变量，包括没有变量

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke")
prompt_template.format()

'Tell me a joke'

### [ChatPromptTemplate](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)
提示聊天模型/是聊天消息的列表。

每个聊天消息与内容相关联，还有一个称为角色的附加参数。例如，在OpenAI聊天完成API中，聊天消息可以与AI助手，人类或系统角色相关联。

In [6]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI bot. Your name is {name}."),
        ("human", "Hello, how are you doing?"),
        ("ai", "I'm doing well, thanks!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")

将这些格式化的消息导入到LangChain的ChatOpenAI聊天模型类中，大致相当于直接使用OpenAI客户端的以下操作

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful AI bot. Your name is Bob."},
        {"role": "user", "content": "Hello, how are you doing?"},
        {"role": "assistant", "content": "I'm doing well, thanks!"},
        {"role": "user", "content": "What is your name?"},
    ],
)

`ChatPromptTemplate.from_messages`接受各种消息表示，并且是一种方便的方法，可以用您想要的消息格式化聊天模型的输入。

例如，除了使用上面使用的(类型、内容)的二元组表示之外，您还可以传入`MessagePromptTemplate`或`BaseMessage`的实例。

In [7]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)
messages = chat_template.format_messages(text="I don't like eating tasty things")
print(messages)

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content="I don't like eating tasty things")]


### Message Prompts
LangChain提供了不同类型的`MessagePromptTemplate`。最常用的是`AIMessagePromptTemplate`, `SystemMessagePromptTemplate`和`HumanMessagePromptTemplate`，它们分别创建AI消息，系统消息和人类消息。您可以在这里阅读有关[不同类型消息的更多信息](https://python.langchain.com/docs/modules/model_io/chat/message_types/)。


在聊天模型支持使用任意角色接收聊天消息的情况下，您可以使用`ChatMessagePromptTemplate`，它允许用户指定角色名称。

In [8]:
from langchain_core.prompts import ChatMessagePromptTemplate

prompt = "May the {subject} be with you"

chat_message_prompt = ChatMessagePromptTemplate.from_template(
    role="Jedi", template=prompt
)
chat_message_prompt.format(subject="force")

ChatMessage(content='May the force be with you', role='Jedi')

### MessagesPlaceholder
LangChain还提供了`MessagesPlaceholder`，它使您可以完全控制在格式化期间呈现哪些消息。当您不确定应该为消息提示模板使用什么角色，或者希望在格式化过程中插入消息列表时，此功能非常有用。

In [14]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

human_prompt = "Summarize our conversation so far in {word_count} words."
human_message_template = HumanMessagePromptTemplate.from_template(human_prompt)

chat_prompt = ChatPromptTemplate.from_messages(
    [MessagesPlaceholder(variable_name="conversation"), human_message_template]
)
chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[MessagesPlaceholder(variable_name='conversation'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['word_count'], template='Summarize our conversation so far in {word_count} words.'))])

In [11]:
from langchain_core.messages import AIMessage, HumanMessage

human_message = HumanMessage(content="What is the best way to learn programming?")
ai_message = AIMessage(
    content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
"""
)

chat_prompt.format_prompt(
    conversation=[human_message, ai_message], word_count="10"
).to_messages()

[HumanMessage(content='What is the best way to learn programming?'),
 AIMessage(content='1. Choose a programming language: Decide on a programming language that you want to learn.\n\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.\n\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience'),
 HumanMessage(content='Summarize our conversation so far in 10 words.')]

In [16]:
# prompt = MessagesPlaceholder("history")
# prompt.format_messages() # raises KeyError

prompt = MessagesPlaceholder("history", optional=True)
prompt.format_messages() # returns empty list []

prompt.format_messages(
    history=[
        ("system", "You are an AI assistant."),
        ("human", "Hello!"),
    ]
)

[SystemMessage(content='You are an AI assistant.'),
 HumanMessage(content='Hello!')]

### LCEL
`PromptTemplate`和`ChatPromptTemplate`实现了`Runnable`接口，这是`LangChain`表达式语言(LCEL)的基本构建块。这意味着它们支持`invoke`, `ainvoke`, `stream`, `aststream`, `batch`, `abbatch`, `aststream`日志调用。

`PromptTemplate`接受一个字典(提示变量的字典)并返回一个`StringPromptValue`。`ChatPromptTemplate`接受一个字典并返回一个`ChatPromptValue`。

In [17]:
prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

prompt_val = prompt_template.invoke({"adjective": "funny", "content": "chickens"})
prompt_val

StringPromptValue(text='Tell me a funny joke about chickens.')

In [18]:
prompt_val.to_string()

'Tell me a funny joke about chickens.'

In [19]:
prompt_val.to_messages()

[HumanMessage(content='Tell me a funny joke about chickens.')]

In [21]:
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "You are a helpful assistant that re-writes the user's text to "
                "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

chat_val = chat_template.invoke({"text": "i dont like eating tasty things."})
chat_val

ChatPromptValue(messages=[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."), HumanMessage(content='i dont like eating tasty things.')])

In [22]:
chat_val.to_messages()

[SystemMessage(content="You are a helpful assistant that re-writes the user's text to sound more upbeat."),
 HumanMessage(content='i dont like eating tasty things.')]

In [23]:
chat_val.to_string()

"System: You are a helpful assistant that re-writes the user's text to sound more upbeat.\nHuman: i dont like eating tasty things."

## [例选择器](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/)


如果您有大量的示例，您可能需要选择在提示符中包含哪些示例。示例选择器是负责这样做的类。

基接口定义如下:


In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, List, Any
class BaseExampleSelector(ABC):
    """Interface for selecting examples to include in prompts."""

    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """Select which examples to use based on the inputs."""
        
    @abstractmethod
    def add_example(self, example: Dict[str, str]) -> Any:
        """Add new example to store."""

它需要定义的唯一方法是一个`select_examples`方法。它接受输入变量，然后返回一个示例列表。如何选择这些示例取决于每个具体实现。

`LangChain`有几种不同类型的示例选择器。有关所有这些类型的概述，请参见下表。

在本指南中，我们将逐步创建一个自定义示例选择器。


### Examples

为了使用示例选择器，我们需要创建一个示例列表。这些通常应该是示例输入和输出。出于演示目的，让我们假设我们正在选择如何将英语翻译成意大利语的示例。

In [26]:
examples = [
    {"input": "hi", "output": "ciao"},
    {"input": "bye", "output": "arrivaderci"},
    {"input": "soccer", "output": "calcio"},
]

### 自定义示例选择器

让我们编写一个示例选择器，它根据单词的长度选择要选择的示例。

In [27]:
from langchain_core.example_selectors.base import BaseExampleSelector


class CustomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        # This assumes knowledge that part of the input will be a 'text' key
        new_word = input_variables["input"]
        new_word_length = len(new_word)

        # Initialize variables to store the best match and its length difference
        best_match = None
        smallest_diff = float("inf")

        # Iterate through each example
        for example in self.examples:
            # Calculate the length difference with the first word of the example
            current_diff = abs(len(example["input"]) - new_word_length)

            # Update the best match if the current one is closer in length
            if current_diff < smallest_diff:
                smallest_diff = current_diff
                best_match = example

        return [best_match]

In [33]:
example_selector = CustomExampleSelector(examples)
example_selector

In [28]:
example_selector = CustomExampleSelector(examples)
example_selector.select_examples({"input": "okay"})

[{'input': 'bye', 'output': 'arrivaderci'}]

In [29]:
example_selector.add_example({"input": "hand", "output": "mano"})
example_selector.select_examples({"input": "okay"})

[{'input': 'hand', 'output': 'mano'}]

In [31]:
example_selector

### Use in a Prompt
现在我们可以在提示符中使用这个示例选择器

In [34]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate.from_template("Input: {input} -> Output: {output}")

example_prompt

PromptTemplate(input_variables=['input', 'output'], template='Input: {input} -> Output: {output}')

In [36]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Input: {input} -> Output:",
    prefix="Translate the following words from English to Italain:",
    input_variables=["input"],
)

print(prompt.format(input="wordss"))

Translate the following words from English to Italain:

Input: soccer -> Output: calcio

Input: wordss -> Output:


### Example Selector Types
| 名称 | 描述 |
| ---------------- | ----------- |
| 相似性 | 使用输入和示例之间的语义相似性来决定选择哪个示例。|
| MMR | 使用输入和示例之间的最大边际相关性来决定选择哪个示例。|
| Length | 选择基于多少可以适合在一定长度的例子|
| Ngram | 使用输入和示例之间的ngram重叠来决定选择哪个示例。|

#### Select by length

这个示例选择器根据长度选择要使用的示例。当您担心构造一个将超过上下文窗口长度的提示时，这很有用。对于较长的输入，它会选择更少的例子，而对于较短的输入，它会选择更多的例子。

In [37]:
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    # The examples it has available to choose from.
    examples=examples,
    # The PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # The maximum length that the formatted examples should be.
    # Length is measured by the get_text_length function below.
    max_length=25,
    # The function used to get the length of a string, which is used
    # to determine which examples to include. It is commented out because
    # it is provided as a default value if none is specified.
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [38]:
# An example with small input, so it selects all examples.
print(dynamic_prompt.format(adjective="big"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output:


In [39]:
# An example with long input, so it selects only one example.
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

Give the antonym of every input

Input: happy
Output: sad

Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:


In [40]:
# You can add an example to an example selector as well.
new_example = {"input": "big", "output": "small"}
dynamic_prompt.example_selector.add_example(new_example)
print(dynamic_prompt.format(adjective="enthusiastic"))

Give the antonym of every input

Input: happy
Output: sad

Input: tall
Output: short

Input: energetic
Output: lethargic

Input: sunny
Output: gloomy

Input: windy
Output: calm

Input: big
Output: small

Input: enthusiastic
Output:


#### [Select by maximal marginal relevance (MMR)](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/mmr/)


`MaxMarginalRelevanceExampleSelector`根据与输入最相似的示例组合来选择示例，同时还优化了多样性。它通过寻找与输入具有最大余弦相似度的嵌入的例子来实现这一点，然后迭代地添加它们，同时惩罚它们与已经选择的例子的接近程度。

In [41]:
from langchain_community.vectorstores import FAISS
from langchain_core.example_selectors import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector,
)

from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# 创造反义词的模拟任务的例子。
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    # 可供选择的示例列表
    examples,
    # 嵌入类用于生成用于度量语义相似度的嵌入.
    OpenAIEmbeddings(),
    # VectorStore类，用于存储嵌入并对其进行相似性搜索.
    FAISS,
    # T生成示例的数量
    k=2,
)


mmr_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

## 输入的是一种感觉，所以应该选择快乐|悲伤的例子作为第一个
print(mmr_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: windy
Output: calm

Input: worried
Output:


让我们将其与仅考虑相似性（使用 `SemanticSimilarityExampleSelector` 而不是 `MaxMarginalRelevanceExampleSelector`）所得到的结果进行比较。

In [42]:
# Let's compare this to what we would just get if we went solely off of similarity,
# by using SemanticSimilarityExampleSelector instead of MaxMarginalRelevanceExampleSelector.
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # The number of examples to produce.
    k=2,
)
similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)
print(similar_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: sunny
Output: gloomy

Input: worried
Output:


#### [Select by n-gram overlap](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/ngram_overlap/)

`NGramOverlapExampleSelector`根据`ngram`重叠分数，根据与输入最相似的示例选择并排序示例。`ngram`重叠分数是介于0.0和1.0之间的浮点数。


In [44]:
from langchain_community.example_selectors.ngram_overlap import (
    NGramOverlapExampleSelector,
)
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a fictional translation task.
examples = [
    {"input": "See Spot run.", "output": "Ver correr a Spot."},
    {"input": "My dog barks.", "output": "Mi perro ladra."},
    {"input": "Spot can run.", "output": "Spot puede correr."},
]

example_selector = NGramOverlapExampleSelector(
    # 提供了可供选择的示例。
    examples=examples,
    # PromptTemplate用于格式化示例
    example_prompt=example_prompt,
    # 选择器停止的阈值。
    # 默认设置为-1.0。
    threshold=-1.0,
    # 对于负阈值：
    #    选择器按 ngram 重叠分数对示例进行排序，并且不排除任何示例。
    # 对于大于 1.0 的阈值：
    #    选择器排除所有示例，并返回一个空列表。
    # 对于等于 0.0 的阈值：
    #    选择器按 ngram 重叠分数对示例进行排序，
    #    并排除那些与输入没有 ngram 重叠的内容。
)
dynamic_prompt = FewShotPromptTemplate(
    #我们提供了一个ExampleSelector而不是examples。
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the Spanish translation of every input",
    suffix="Input: {sentence}\nOutput:",
    input_variables=["sentence"],
)
# An example input with large ngram overlap with "Spot can run."
# and no overlap with "My dog barks."
print(dynamic_prompt.format(sentence="Spot can run fast."))

Give the Spanish translation of every input

Input: Spot can run.
Output: Spot puede correr.

Input: See Spot run.
Output: Ver correr a Spot.

Input: My dog barks.
Output: Mi perro ladra.

Input: Spot can run fast.
Output:


In [45]:
# 你也可以向NGramOverlapExampleSelector中添加示例。
new_example = {"input": "Spot plays fetch.", "output": "Spot juega a buscar."}

example_selector.add_example(new_example)
print(dynamic_prompt.format(sentence="Spot can run fast."))

Give the Spanish translation of every input

Input: Spot can run.
Output: Spot puede correr.

Input: See Spot run.
Output: Ver correr a Spot.

Input: Spot plays fetch.
Output: Spot juega a buscar.

Input: My dog barks.
Output: Mi perro ladra.

Input: Spot can run fast.
Output:


您可以设置排除示例的阈值。
例如，设置阈值等于`0.0`
排除与输入没有 `ngram` 重叠的示例。
如 "My dog barks."与"Spot can run fast"没有 `ngram` 重叠。
它被排除在外。

In [46]:
example_selector.threshold = 0.0
print(dynamic_prompt.format(sentence="Spot can run fast."))

Give the Spanish translation of every input

Input: Spot can run.
Output: Spot puede correr.

Input: See Spot run.
Output: Ver correr a Spot.

Input: Spot plays fetch.
Output: Spot juega a buscar.

Input: Spot can run fast.
Output:


设置小的非零阈值

In [47]:
# Setting small nonzero threshold
example_selector.threshold = 0.09
print(dynamic_prompt.format(sentence="Spot can play fetch."))

Give the Spanish translation of every input

Input: Spot can run.
Output: Spot puede correr.

Input: Spot plays fetch.
Output: Spot juega a buscar.

Input: Spot can play fetch.
Output:


阈值设置大于1.0

In [48]:
# Setting threshold greater than 1.0
example_selector.threshold = 1.0 + 1e-9
print(dynamic_prompt.format(sentence="Spot can play fetch."))

Give the Spanish translation of every input

Input: Spot can play fetch.
Output:


#### [Select by similarity](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/similarity/)

该对象根据与输入的相似性选择示例。它通过寻找与输入具有最大余弦相似度的嵌入的例子来做到这一点。

In [49]:
from langchain_chroma import Chroma 
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 可供选择的示例列表。
    examples,
    # 嵌入类用于生成用于度量语义相似度的嵌入
    OpenAIEmbeddings(),
    # 嵌入类用于生成用于度量语义相似度的嵌入
    Chroma,
    # 生成示例的数量。
    k=1,
)
similar_prompt = FewShotPromptTemplate(
    # 我们提供了一个ExampleSelector而不是examples。
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)
# Input is a feeling, so should select the happy/sad example
print(similar_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:


输入是一种测量，因此选择高/简短的例子

In [50]:
# Input is a measurement, so should select the tall/short example
print(similar_prompt.format(adjective="large"))

Give the antonym of every input

Input: tall
Output: short

Input: large
Output:


您还可以向`SemanticSimilarityExampleSelector`添加新的示例

In [51]:

similar_prompt.example_selector.add_example(
    {"input": "enthusiastic", "output": "apathetic"}
)
print(similar_prompt.format(adjective="passionate"))

Give the antonym of every input

Input: enthusiastic
Output: apathetic

Input: passionate
Output:


### [聊天模型的几个例子](https://python.langchain.com/docs/modules/model_io/prompts/few_shot_examples_chat/)


本手册介绍了如何在聊天模型中使用几个示例。对于如何最好地执行少量提示，似乎还没有达成坚实的共识，并且最佳提示编译可能因模型而异。正因为如此，我们提供了像[`FewShotChatMessagePromptTemplate`](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate.html?highlight=fewshot#langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate)这样的短提示模板，作为一个灵活的起点，您可以根据需要修改或替换它们。

几次提示模板的目标是根据输入动态选择示例，然后在最后的提示中格式化示例以提供给模型。

注意:下面的代码示例适用于聊天模型。对于完成模型(llm)类似的少样本提示示例，请参阅[few-shot prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/few_shot_examples/)指南。

#### 固定例子
最基本(也是最常见)的几次提示技术是使用固定提示示例。通过这种方式，您可以选择一个链条，对其进行评估，并避免担心生产中额外的活动部件。

模板的基本组件是:`-examples`:在最后提示符中包含的字典示例列表。—示例提示符:通过[`format_messages`](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html?highlight=format_messages#langchain_core.prompts.chat.ChatPromptTemplate.format_messages)方法将每个示例转换为1个或多个消息。一个常见的例子是将每个示例转换为一个人工消息和一个人工智能消息响应，或者一个人工消息后跟一个函数调用消息。

下面是一个简单的演示

In [52]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


最后，组装最终的提示符并将其与模型一起使用。

In [53]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

from langchain_community.chat_models import ChatAnthropic

chain = final_prompt | ChatAnthropic(temperature=0.0)

chain.invoke({"input": "What's the square of a triangle?"})

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

#### Dynamic few-shot prompting
有时，您可能希望根据输入来限定显示哪些示例。为此，您可以用`example_selector`替换`examples` 。其他组件与上面相同!回顾一下，动态的少量提示模板看起来是这样的

- `example_selector`: 负责为给定输入选择少量示例(以及它们返回的顺序)。这些实现了[`BaseExampleSelector`](https://api.python.langchain.com/en/latest/example_selectors/langchain_core.example_selectors.base.BaseExampleSelector.html?highlight=baseexampleselector#langchain_core.example_selectors.base.BaseExampleSelector)接口。一个常见的例子是支持向量存储的[SemanticSimilarityExampleSelector](https://api.python.langchain.com/en/latest/example_selectors/langchain_core.example_selectors.semantic_similarity.SemanticSimilarityExampleSelector.html?highlight=semanticsimilarityexampleselector#langchain_core.example_selectors.semantic_similarity.SemanticSimilarityExampleSelector)

- `example_prompt`:将每个示例转换为1个或多个消息
- [`format_messages`](https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html?highlight=chatprompttemplate#langchain_core.prompts.chat.ChatPromptTemplate.format_messages)方法。一个常见的例子是将每个示例转换为一个人工消息和一个人工智能消息响应，或者一个人工消息后跟一个函数调用消息。

同样，这些可以与其他消息和聊天模板组合在一起，以组装最终提示。

In [ ]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

由于我们使用向量库根据语义相似度选择示例，因此我们希望首先填充该库。

In [54]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

to_vectorize = [" ".join(example.values()) for example in examples]
to_vectorize

['2+2 4',
 '2+3 5',
 '2+4 6',
 'What did the cow say to the moon? nothing at all',
 'Write me a poem about the moon One for the moon, and one for me, who are we to talk about the moon?']

In [55]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

创建示例选择器。创建vectorstore后，您可以创建示例选择器。这里我们将指示它只获取前2个示例。

In [56]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
example_selector.select_examples({"input": "horse"})

[{'input': 'What did the cow say to the moon?', 'output': 'nothing at all'},
 {'input': 'windy', 'output': 'calm'}]

使用上面创建的示例选择器组装提示模板。

In [57]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

# Define the few-shot prompt.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ),
)
print(few_shot_prompt.format(input="What's 3+3?"))

Human: 2+3
AI: 5
Human: 2+2
AI: 4


组装最终的提示模板

In [58]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wondrous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(few_shot_prompt.format(input="What's 3+3?"))

Human: 2+3
AI: 5
Human: 2+2
AI: 4


#### 与LLM一起使用


In [60]:
from langchain_community.chat_models import ChatOpenAI

chain = final_prompt | ChatOpenAI(temperature=0.0)

chain.invoke({"input": "What's 3+3?"})

AIMessage(content='6', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 51, 'total_tokens': 52}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-faf40c03-ba64-4710-80c5-6f6c37336876-0')

### [Few-shot prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/few_shot_examples/)

在本教程中，我们将学习如何创建一个使用少量示例的提示模板。可以从一组示例或从一个示例选择器对象构造一个简短的提示模板。


使用一个示例集     
首先，创建一个由几个示例组成的列表。每个示例都应该是一个字典，其中键是输入变量，值是这些输入变量的值。

In [61]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

##### 为几个示例创建一个格式化程序
配置一个格式化器，将少数几个示例格式化为字符串。这个格式化程序应该是一个   `PromptTemplate`对象。

In [62]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], 
    template="Question: {question}\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



##### 向`FewShotPromptTemplate`提供示例和格式化程序
最后，创建一个`FewShotPromptTemplate`对象。该对象接受少数镜头示例和少数镜头示例的格式化程序。

In [63]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

#### 使用示例选择器
##### 将示例提供给`ExampleSelector`

我们将重用上一节中的示例集和格式化器。然而，我们不是将示例直接提供给`FewShotPromptTemplate`对象，而是将它们提供给一个`ExampleSelector`对象。

在本教程中，我们将使用`SemanticSimilarityExampleSelector`类。这个类根据与输入的相似度选择几个镜头示例。它使用嵌入模型来计算输入和少数镜头示例之间的相似性，并使用向量存储来执行最近邻搜索。


In [64]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 这是可供选择的示例列表。.
    examples,
    # 这是一个嵌入类，用于生成用于度量语义相似度的嵌入
    OpenAIEmbeddings(),
    # 这是VectorStore类，用于存储嵌入并进行相似性搜索
    Chroma,
    # 这是要生成的样本的数量
    k=1,
)

# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

Examples most similar to the input: Who was the father of Mary Ball Washington?


answer: 
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball

question: Who was the maternal grandfather of George Washington?


#### 将示例选择器提供给`FewShotPromptTemplate`
最后，创建一个`FewShotPromptTemplate`对象。该对象接受示例选择器和少数镜头示例的格式化器。

In [65]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


Question: Who was the father of Mary Ball Washington?


### 部分提示模板
像其他方法一样，对提示模板进行局部化是有意义的——例如，传入所需值的子集，以创建一个只需要剩余值子集的新提示模板。

LangChain以两种方式支持这一点:使用字符串值进行部分格式化。2. 使用返回字符串值的函数进行部分格式化。

这两种方式支持不同的用例。在下面的示例中，我们将讨论这两个用例的动机以及如何在LangChain中实现它。

#### 带字符串的偏函数
想要局部化提示模板的一个常见用例是，您在其他变量之前获得了一些变量。例如，假设您有一个提示模板，它需要两个变量foo和baz。如果您在链的早期获得foo值，但稍后获得baz值，那么等到两个变量位于同一位置时才将它们传递给提示模板可能会很烦人。相反，您可以使用foo值对提示模板进行局部化，然后传递局部化的提示模板，然后使用它。下面是这样做的一个例子

In [66]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("{foo}{bar}")
partial_prompt = prompt.partial(foo="foo")
print(partial_prompt.format(bar="baz"))



foobaz


您还可以使用部分变量初始化提示符。

In [67]:
prompt = PromptTemplate(
    template="{foo}{bar}", input_variables=["bar"], partial_variables={"foo": "foo"}
)
print(prompt.format(bar="baz"))

foobaz


#### 带函数偏导
另一个常见的用法是对函数进行偏导。这个用例是当你有一个变量，你知道你总是想以一种常见的方式获取。最典型的例子就是日期或时间。假设您有一个提示符，您总是希望显示当前日期。您不能在提示符中硬编码它，并且将它与其他输入变量一起传递有点烦人。在这种情况下，使用一个总是返回当前日期的函数对提示符进行局部化是非常方便的。

In [68]:
from datetime import datetime


def _get_datetime():
    now = datetime.now()
    return now.strftime("%m/%d/%Y, %H:%M:%S")


prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)
partial_prompt = prompt.partial(date=_get_datetime)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 04/30/2024, 10:40:38


您还可以使用部分变量初始化提示，这在此工作流中通常更有意义。

In [69]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective"],
    partial_variables={"date": _get_datetime},
)
print(prompt.format(adjective="funny"))

Tell me a funny joke about the day 04/30/2024, 10:41:00


### 成分
LangChain提供了一个用户友好的界面，可以将不同部分的提示组合在一起。您可以使用字符串提示或聊天提示来完成此操作。以这种方式构造提示可以方便地重用组件。
##### 字符串提示组成
当使用字符串提示时，每个模板都连接在一起。可以直接使用提示符，也可以使用字符串(列表中的第一个元素必须是提示符)。




In [72]:
from langchain_core.prompts import PromptTemplate



In [71]:
PromptTemplate.from_template("Tell me a joke about {topic}")

PromptTemplate(input_variables=['topic'], template='Tell me a joke about {topic}')

In [73]:
prompt = (
    PromptTemplate.from_template("Tell me a joke about {topic}")
    + ", make it funny"
    + "\n\nand in {language}"
)
prompt

PromptTemplate(input_variables=['language', 'topic'], template='Tell me a joke about {topic}, make it funny\n\nand in {language}')

In [74]:
prompt.format(topic="sports", language="spanish")

'Tell me a joke about sports, make it funny\n\nand in spanish'

您也可以在LLMChain中使用它，就像以前一样。

In [75]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
chain = LLMChain(llm=model, prompt=prompt)
chain.run(topic="sports", language="spanish")

'¿Por qué los futbolistas son tan buenos escalando montañas? Porque siempre están en fuera de juego.'

#### 聊天提示组成

聊天提示由消息列表组成。纯粹为了开发人员的体验，我们添加了一种方便的方法来创建这些提示符。在这个管道中，每个新元素都是最终提示符中的新消息。



In [76]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage




首先，让我们用一条系统消息初始化基本的`ChatPromptTemplate`。它不一定要从系统开始，但它通常是一个很好的实践

In [77]:
prompt = SystemMessage(content="You are a nice pirate")

然后，您可以轻松地创建将其与其他消息或消息模板组合在一起的管道。当没有要格式化的变量时使用`Message`，当有要格式化的变量时使用`MessageTemplate`。你也可以只使用一个字符串(注意:这将自动被推断为一个`HumanMessagePromptTemplate`)。

In [79]:
new_prompt = (
    prompt + HumanMessage(content="hi") + AIMessage(content="what?") + "{input}"
)
new_prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessage(content='You are a nice pirate'), HumanMessage(content='hi'), AIMessage(content='what?'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

在底层，它创建了`ChatPromptTemplate`类的一个实例，因此您可以像以前一样使用它

In [80]:
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate'),
 HumanMessage(content='hi'),
 AIMessage(content='what?'),
 HumanMessage(content='i said hi')]

您也可以在LLMChain中使用它，就像以前一样。

In [81]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
model = ChatOpenAI()
chain = LLMChain(llm=model, prompt=new_prompt)
chain.run("i said hi")

'Oh, hi! How can I help you today?'

### 使用PipelinePrompt
LangChain包含了一个抽象的`PipelinePromptTemplate`，当你想重用部分提示时，它会很有用。`PipelinePrompt`由两个主要部分组成
- 最终提示符:返回的最终提示符
- 管道提示符:一个元组列表，由字符串名称和提示模板组成。每个提示模板将被格式化，然后作为具有相同名称的变量传递给未来的提示模板。



In [82]:
from langchain_core.prompts.pipeline import PipelinePromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

full_template = """{introduction}

{example}

{start}"""
full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """Here's an example of an interaction:

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)
pipeline_prompt.input_variables

['example_q', 'example_a', 'person', 'input']

In [83]:
print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)

You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:
